In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
from polire.interpolate import  Kriging, Idw
import sys
import gpflow
import tensorflow as tf

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

In [ ]:
df = pd.read_csv("../data/beijinga_scaled.csv")
df = df.loc[:, ~df.columns.str.match('Unnamed')]
df = df.rename(columns={'ts': 'Time', 'station_id': 'Station'})
timestamps = df['Time'].unique()
timestamps.sort()
stations = df['Station'].unique()
stations.sort()


In [ ]:
df.head()

In [ ]:
stations = df['Station'].unique()
stations.sort()


In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 6, shuffle = True, random_state = 0)
timestamps = df['Time'].unique()
timestamps.sort()

In [ ]:
k = 30

In [ ]:
splits = 6

In [ ]:
rmse = {'GP':{timestamps[i]:{i:np.nan for i in range(splits)} for i in range(k, len(timestamps))}, 'IDW':{timestamps[i]:{i:np.nan for i in range(splits)} for i in range(k, len(timestamps))}, 'Krig':{timestamps[i]:{i:np.nan for i in range(splits)} for i in range(k, len(timestamps))}}
mae = {'GP':{timestamps[i]:{i:np.nan for i in range(splits)} for i in range(k, len(timestamps))}, 'IDW':{timestamps[i]:{i:np.nan for i in range(splits)} for i in range(k, len(timestamps))}, 'Krig':{timestamps[i]:{i:np.nan for i in range(splits)} for i in range(k, len(timestamps))}}

In [ ]:


for idx in range(k, len(timestamps)):
    
    print(idx, timestamps[idx])
    timestamp = timestamps[idx]

    i = -1 

    for train_index, test_index in kf.split(stations):
        
        
        i = i + 1
        train_stations = stations[train_index]
        test_stations = stations[test_index]
        try:
            train_df = pd.concat([df.groupby('Station').get_group(stn) for stn in train_stations])
            train_df = train_df.groupby('Time').get_group(timestamp)
            train_df.drop(columns = ['Station', 'Time'])
            test_df = pd.concat([df.groupby('Station').get_group(stn) for stn in test_stations])
            test_df = test_df.groupby('Time').get_group(timestamp)
            test_df.drop(columns = ['Station', 'Time'])

        except KeyError:
            print("No train or test data for,", timestamp)
            continue


#         X_train = np.array(train_df[['longitude', 'latitude']])
#         y_train = np.array(train_df[['PM2.5']])
#         X_test = np.array(test_df[['longitude', 'latitude']])
#         y_test = np.array(test_df[['PM2.5']])

#         idw = Idw()
#         idw.fit(X_train, y_train)
#         krig = Kriging()
#         krig.fit(X_train, y_train)
        
        gp_train_df = []
        last_k = timestamps[idx - k + 1: idx + 1]
        
        for station in train_stations:
            station_df = df.groupby('Station').get_group(station)
            
            for tme in last_k:
                try:
                    temp = station_df.groupby('Time').get_group(tme)
                    gp_train_df.append(temp)
                except KeyError:
                    continue
                
        tf.reset_default_graph()
        graph = tf.get_default_graph()
        gpflow.reset_default_session(graph=graph)

        try:


            xy_matern_1 = gpflow.kernels.Matern32(input_dim=2, ARD=True, active_dims=[0, 1])
            xy_matern_2 = gpflow.kernels.Matern32(input_dim=2, ARD=True, active_dims=[0, 1])

            t_matern = gpflow.kernels.Matern32(input_dim=1, active_dims=[2])
            t_other = gpflow.kernels.Matern32(input_dim=1, active_dims=[2])*gpflow.kernels.Periodic(input_dim=1, active_dims=[2])+\
            gpflow.kernels.Matern32(input_dim=1, active_dims=[2])*gpflow.kernels.Periodic(input_dim=1, active_dims=[2])+\
            gpflow.kernels.Matern32(input_dim=1, active_dims=[2])*gpflow.kernels.Periodic(input_dim=1, active_dims=[2])+\
            gpflow.kernels.Matern32(input_dim=1, active_dims=[2])*gpflow.kernels.Periodic(input_dim=1, active_dims=[2])+\
            gpflow.kernels.Matern32(input_dim=1, active_dims=[2])*gpflow.kernels.Periodic(input_dim=1, active_dims=[2])

            time = t_matern + t_other
            overall_kernel = (xy_matern_1 + xy_matern_2) * time

            gp_train_df = pd.concat(gp_train_df)
            X_train_gp = np.array(gp_train_df[['longitude', 'latitude', 'Time']])
            y_train_gp = np.array(gp_train_df[['PM2.5']])


            model = gpflow.models.GPR(X_train_gp, y_train_gp, kern = overall_kernel, mean_function = None)
            opt = gpflow.train.ScipyOptimizer()
            opt.minimize(model)

            mean, variance = model.predict_y(X_test)


            gp_rmse_error = np.sqrt(mean_squared_error(mean, y_test))
            gp_mae_error = mean_absolute_error(mean, y_test)

#             krig_rmse_error = np.sqrt(mean_squared_error(krig.predict(X_test), y_test))
#             krig_mae_error = mean_absolute_error(krig.predict(X_test), y_test)

#             idw_rmse_error = np.sqrt(mean_squared_error(idw.predict(X_test), y_test))
#             idw_mae_error = mean_absolute_error(idw.predict(X_test), y_test)
            
        except Exception as e:
            print(e)
            continue
        
        rmse['GP'][timestamp][i] = gp_rmse_error
        mae['GP'][timestamp][i] = gp_mae_error
        
#         rmse['Krig'][timestamp][i] = krig_rmse_error
#         mae['Krig'][timestamp][i] = krig_mae_error
        
#         rmse['IDW'][timestamp][i] = idw_rmse_error
#         mae['IDW'][timestamp][i] = idw_mae_error
        
        print(timestamp, "-----", "GP", "-----", gp_rmse_error)
#               , "\nIDW -------", idw_rmse_error, "\nKrig-----", krig_rmse_error)

30 0.08516483516483518
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 3249.665837
  Number of iterations: 47
  Number of functions evaluations: 56


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 3249.665837
  Number of iterations: 47
  Number of functions evaluations: 56


0.08516483516483518 ----- GP ----- 18.72674205531324
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 3242.210634
  Number of iterations: 118
  Number of functions evaluations: 231


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 3242.210634
  Number of iterations: 118
  Number of functions evaluations: 231


0.08516483516483518 ----- GP ----- 14.882167114400708
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 3270.600969
  Number of iterations: 46
  Number of functions evaluations: 64


...

0.5906593406593407 ----- GP ----- 45.20899916902248
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 5353.144740
  Number of iterations: 25
  Number of functions evaluations: 56


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 5353.144740
  Number of iterations: 25
  Number of functions evaluations: 56


0.5906593406593407 ----- GP ----- 45.20900028191829
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 4342.121907
  Number of iterations: 67
  Number of functions evaluations: 104


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 4342.121907
  Number of iterations: 67
  Number of functions evaluations: 104


0.5906593406593407 ----- GP ----- 45.20896440136834


In [15]:
rmse

{'GP': {0.08516483516483518: {0: 28.693304311223,
   1: 24.31802362863328,
   2: 27.23467104059764,
   3: 17.275651197919984,
   4: 24.89621693050585,
   5: 21.159759535467117},
  0.08791208791208792: {0: 7.19881098700594,
   1: 10.320260812492446,
   2: 9.718980511625047,
   3: 5.6749850795506305,
   4: 9.404005524249023,
   5: 6.461185246250385},
  0.09065934065934066: {0: 35.39035882505555,
   1: 23.013064770114095,
   2: 26.153353959012566,
   3: 8.824879703583434,
   4: 20.60826829627894,
   5: 10.300618973489744},
  0.0934065934065934: {0: 17.788938779051673,
   1: 32.46995412879435,
   2: 17.192147254469166,
   3: 13.751945570890372,
   4: 34.29496408814983,
   5: 33.36711247243725},
  0.09615384615384616: {0: 10.451288082457381,
   1: 44.322758008075205,
   2: 15.15968837842309,
   3: 10.217305592823923,
   4: 31.19201189596954,
   5: 23.750292440048288},
  0.0989010989010989: {0: 20.577631947337764,
   1: 28.363868477293746,
   2: 33.32262179132606,
   3: 22.61492871909364,
  

In [ ]:
a = 100

In [ ]:
X_train_gp[:,2]

In [ ]:
timestamp

In [ ]:
train_stations


In [ ]:
df[~df['Station'].isin(train_stations)].groupby('Time').get_group(timestamp)


In [ ]:
t1 = df[df['Station'].isin(train_stations)].groupby('Time').get_group(timestamp)
t2 = df[~df['Station'].isin(train_stations)].groupby('Time').get_group(timestamp)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(t1['longitude'], t1['latitude'], t1['PM2.5'], alpha = 0.2)
plt.figure()
plt.scatter(t2['longitude'], t2['latitude'], t2['PM2.5'], alpha = 0.2)

In [ ]:
df[df['Station'].isin(test_stations)].groupby('Time').get_group(timestamp)

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
      # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e:
        # Visible devices must be set at program startup
        print(e)
